In [ ]:
from tensorflow.keras.layers import Input,Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.datasets import mnist
import numpy as np

In [ ]:
batch_size = 100
original_dim = 784
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

In [ ]:
def sampling(args: tuple):
  z_mean, z_log_var = args
  epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),mean=0.,stddev=epsilon_std)
  return z_mean+K.exp(z_log_var/2)*epsilon

x = Input(shape=(original_dim,),name = "input")
h = Dense(intermediate_dim, activation="relu",name="encoding")(x)
z_mean = Dense(latent_dim, name="mean")(h)
z_log_var = Dense(latent_dim, name="log-variance")(h)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean,z_log_var])
encoder = Model(x,[z_mean,z_log_var,z], name="encoder")


In [ ]:
input_decoder = Input(shape=(latent_dim,), name="decoder_input")
decoder_h = Dense(intermediate_dim, activation="relu", name="decoder_h")(input_decoder)
x_decoded = Dense(original_dim, activation="sigmoid", name="flat_decoded")(decoder_h)
decoder = Model(input_decoder,x_decoded,name="decoder")


In [ ]:
output_combined = decoder(encoder(x)[2])
vae = Model(x,output_combined)
vae.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [ ]:
kl_loss = -0.5*K.sum(1+z_log_var - K.exp(z_log_var)-K.square(z_mean),axis=-1)
vae.add_loss(K.mean(kl_loss)/784.)
vae.compile(optimizer="rmsprop",loss="binary_crossentropy")

In [ ]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train = x_train.reshape((len(x_train),np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test),np.prod(x_test.shape[1:])))

In [ ]:
vae.fit(x_train,x_train,shuffle=True,epochs=epochs,batch_size=batch_size)

Epoch 1/50
600/600 [==============================] - 10s 16ms/step - loss: 0.2447
Epoch 2/50
600/600 [==============================] - 9s 16ms/step - loss: 0.2167
Epoch 3/50
600/600 [==============================] - 10s 16ms/step - loss: 0.2110
Epoch 4/50
600/600 [==============================] - 9s 16ms/step - loss: 0.2075
Epoch 5/50
600/600 [==============================] - 9s 15ms/step - loss: 0.2051
Epoch 6/50
600/600 [==============================] - 9s 15ms/step - loss: 0.2032
Epoch 7/50
600/600 [==============================] - 9s 16ms/step - loss: 0.2016
Epoch 8/50
600/600 [==============================] - 10s 16ms/step - loss: 0.2003
Epoch 9/50
600/600 [==============================] - 9s 16ms/step - loss: 0.1993
Epoch 10/50
600/600 [==============================] - 9s 16ms/step - loss: 0.1984
Epoch 11/50
600/600 [==============================] - 9s 15ms/step - loss: 0.1978
Epoch 12/50
600/600 [==============================] - 9s 16ms/step - loss: 0.1972
Epoch 13/5